In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
from ta.momentum import StochRSIIndicator
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
today = datetime.date.today()
#yesterday = today - datetime.timedelta(days=1)
dow=today.weekday()

if dow < 5: 
    dte=5-dow
else: 
    dte=12-dow
#print(dte)
friday = str(today + datetime.timedelta( (4-today.weekday()) % 7 ))
start_date="2010-01-01"
end_date=today

#for i in range(0,len(sip)):
def st_rsi():
    df_st_rsi=pd.DataFrame()
    for i in range(0,len(sip)):
        stk=sip[i]
        dfw = yf.download(stk, start_date, end_date,interval='1wk').dropna()
        dfm = yf.download(stk, start_date, end_date,interval='1mo').dropna()
        #dfd = yf.download(stk, yesterday, today,interval='1h').dropna()
        
        stoch_rsi_w=StochRSIIndicator(close=dfw.Close,window=14,smooth1=3, smooth2=3)
        stoch_rsi_m=StochRSIIndicator(close=dfm.Close,window=14,smooth1=3, smooth2=3)


        dfw["kw"]=stoch_rsi_w.stochrsi_k()
        dfm["km"]=stoch_rsi_m.stochrsi_k()
        dfw["Chng"]=(dfw.Close-dfw.Open)/dfw.Open
        STD="{:.2%}".format(np.std(dfw.Chng))
       
        #print(dfm)
        #print(dfw)
        lw=len(dfw.iloc[:,0])
        lm=len(dfm.iloc[:,0])
        #ld=len(dfd.iloc[:,0])
        
        try:
            last=dfw['Close'][lw-1]
            kw="{:.2%}".format(dfw.iloc[lw-1,6])
            km="{:.2%}".format(dfm.iloc[lm-1,6])
            #print(dfd.iloc[ld-1,:])
        except:
            last="na"
            kw="na"
            km="na"
        

        #print(kw,km)
        temp=pd.Series([stk,last,STD,kw,km])
        #print(temp)
        df_st_rsi=df_st_rsi.append(temp,ignore_index=True)
    df_st_rsi.columns=['stk','last','STD','kw','km']
    return(df_st_rsi)

def get_call():
    df_call=pd.DataFrame()
    for i in range(0,len(sip)):
        try:
            stk=sip[i]
            tick= yf.Ticker(stk)
            ch=tick.option_chain(date=friday).calls
            ch=ch[ch.inTheMoney==False]
            #print(ch)
            #ln=len(ch.iloc[:,0])
            yld="{:.2%}".format((ch.iloc[0,3])/ch.iloc[0,2])
            yld_adj="{:.2%}".format((ch.iloc[0,3])/dte*7/ch.iloc[0,2])
            strike=ch.iloc[0,2]
            premium=ch.iloc[0,3]
            premium_adj=ch.iloc[0,3]/dte*7
            temp=pd.Series([stk,strike,premium,premium_adj,yld,yld_adj])
            df_call=df_call.append(temp,ignore_index=True)
        except:
            pass
    df_call.columns=['stk','strike','premium','premium_adj.','yield','yield_adj']
    return(df_call)




f1=st_rsi()
#print(f1)
f2=get_call()
#print(f2)
#print(f1)
f3=f1.set_index('stk').join(f2.set_index('stk'))
#print(df)
print(f3)
f3.to_csv('trade_matrix.csv')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********